# Documentation for Havsim simulation code
$\textbf{Note:}$ This document is not a substitute for reading the actual code documentation. Rather, here we discuss issues which are outside the scope of the actual module documentation. We give some overview of traffic simulation, including some basic of traffic flow theory, and explain some design patterns of the simulation code. We also discuss issues such as code style, installation, etc. which are relevant for contributing to havsim.

# Installation
1. Download havsim from github
2. Open python console, navigate to source folder (the one with setup.py)
3. run 'pip install -e .'  -e means install in developer mode, . means to use the current folder

To check if the package is installed, you can run 'pip list' and check that it is listed. If so, you know the package has been installed. If it's not, it might be because you are using the wrong environment; assuming you are using the anaconda distribution of python, try 'conda info --envs' to see all your environments and the current one you have activated.
To work on the package, you'll also need an IDE (I recommend either pycharm or spyder) and git or github desktop.

# Code Style
We are following the google style guide for python https://google.github.io/styleguide/pyguide.html, which is more or less equivalent to the pep8 style guide. All code which will be part of modules should follow those code styles. No need to worry about code style for scripts. Some older modules are not yet compliant with the code style (for examples of the proper style, see simulation.simulation or simulation.models). <br>
The only exceptions to following the google code style is that we use 110 characters as the maximum line length, and we don't have to use snake case for attribute/variable names. You can also use 'modifiers' which are basically common parts of a variable/attribute name which are at the beginning or end of the name. For example (this list is not exhaustive)- <br>
modifiers at beginning of variable name: max, min, cur (reads as current), new, prev, init, start, end, temp <br>
modifiers at end of variable name: ind (reads as index), mem (reads as memory), dict, list, bounds, len, time, info <br>
So for example instead of cur_name, just say curname. or instead of start_time, just say starttime. This isn't to say you should avoid using snake case with modifiers; rather, it means you the option to not have to type underscores if the meaning of the name is obvious.

To check for code style, you should use pylint. The default settings are mostly fine, I just added a few extra codes to the ignore: invalid-name, W0105, R0912, R0913, R0914, R0915, R0902, R0911, R1705, R1710. Pylint will also return results for refactoring, warnings, and errors, and these can be ignored. <br>
In spyder, it has options to automatically generate docstrings in google format, and can give inline, real time warnings on code style using pycodestyle and pydocstyle. The inline warnings are not a replacement for using pylint but are more convenient.


#  Overview of a Microsimulation 
Vehicles have two models. The first is a car following model, which controls how to update the vehicle's position (think mile marker on the road). They are called car following models because they require your lead vehicle, the vehicle directly in front of you, as an input. The most common type of model will output an acceleration, and requires as input your current position/speed, and your leader's current position/speed. Thus at every timestep, your position and speed are already defined, your current acceleration is obtained from the car following model, and the position and speed in the next timestep are updated based on the acceleration and the discretization scheme used (and so the process repeats). <br> 
The second model type is a lane changing model, which controls how vehicles change lanes. Whereas the car following model requires your lead vehicle, a lane changing model will require the vehicles in neighboring lanes in order to be evaluated. The result of the lane changing model will be either a command to change lanes, or a command to stay in the current lane. Vehicles may be in a discretionary state, (where they change lanes only to try to improve their position) or a mandatory state, where they have to change lanes in order to stay on their predefined route. More complex Lane changing models may involve additional components where vehicles cooperate in order to facilitate lane changing (imagine you see a vehicle trying to merge into your lane, and you slow down so that it can change successfully), or where vehicles modify their car following behavior (imagine becoming more aggresively in order to change into the correct lane so you don't miss your exit/turn). Combined, the car following and lane changing models define how vehicles behave under normal circumstances.  <br> 
Microsimulations require a road network on which the vehicles interact. The road network needs to define which roads connect to which, and different road network types will lead to different vehicle behaviors. Vehicles can also have routes, which are sequences of roads which they want to take. One very important part of a road network are the boundary conditions. There are downstream boundary conditions (downstream meaning in the same direction of travel as the vehicles), which basically replace the car following model after it's leader leaves the simulation area (Recall that we require a lead vehicle to use the car following model, so if the lead vehicle has left the simulation area, there is no way to update it's follower, which is still inside the simulation area). Although the downstream boundary conditions may only be used a short time for any given vehicle, they are still very important because they control how quickly vehicles can exit the road network. There are also upstream boundary conditions, which control when vehicles enter the road network, and initial conditions vehicles enter with (for a car following model giving acceleration, the initial speed of the vehicle needs to be defined). Although simpler to describe, the upstream boundary conditions are more complicated to implement correctly. <br> <br>
### Equilibrium solutions and the Fundamental Diagram
An important concept of car following models which is key to understanding their behavior is that of the equilibrium solution. We won't give a technical definition here, but rather an intuitive one. Essentially, if a lead vehicle maintains a constant speed, the equilibrium solution states that the following vehicle should maintain a constant headway (the distance from the tip of the follower to the end of the leader should be constant). Thus, since the leader has a constant speed, and the follower a constant headway, both vehicles have 0 acceleration, and the same speed. This pair of (speed, headway) defines a particular equilibrium solution for the car following model. Thus, if you give any particular value for speed (headway), the equilibrium solution will give back the matching headway (speed) to obtain the pair of (speed, headaway) which gives the particular equilibrium solution. If you assume vehicles behave rationally, then as the speed increases, the headway should increase as well. A simple way to think of a car following model is that they constantly 'push' vehicles towards the equilibrium solution corresponding to their leader's speed. Of course, if the leader's speed is changing, then the 'push' of the car following model is constantly shifting towards different equilibrium solutions (this is an intuitive explanation as to why car following models can lead to oscillatory behavior). <br> <br>
An important concept for traffic flow theory is that of the fundamental diagram. First, we need to define the meaning of 'flow' and 'density' for traffic. Flow is the number of vehicles passing a point in space per unit time (units of vehicles/time). One simple way to measure flow would be to sit at the side of a road, and count vehicles as they pass. Then the flow would be how many vehicles you count in some duration of time. Density is a measure of how close vehicles are together; it has units of vehicles/space. A simple way to measure density would be to take a picture of a road, and count the number of vehicles on the road, and divide that by the total length of the road. Note also that from the units, it's clear that flow/density = speed. The fundamental diagram defines a relationship between flow and density. Fundamental diagrams have the general shape of a 'bump' - at low densities, the speed is high, but flow is low, because there are hardly any vehicles. At very high densities, both the flow and speed are low, because cars are packed together too closely (i.e. bumper to bumper traffic). The maximum flow is achieved at some intermediate density, where vehicles can still travel with high speeds. Everything to the left of the maximum flow is refered to as the free flow branch of the fundamental diagram, because vehicles are still maintaining high speeds. To the right of the maximum flow is the congested branch, where densities are high and speeds are low. Note that for any flow, there will be two possible densities - one corresponding to free flow and one to congested conditions. Also note that equilibrium solutions can be converted to a fundamental diagram, by converting headway into density, and then using flow = density * speed. 
### Traffic oscillations and traffic waves
The concepts of traffic oscillations and traffic waves are closely related to the current understanding of congestion. One should think of traffic waves in the context of the fundamental diagram. Imagine traffic as being a fluid, like the water in a river. The riverbed is like the road, with the water representing vehicles. A region of traffic having high density (i.e. a congested region) is like a wave of water. Similarly to how a wave travels along the length of the body of water, the congested region of traffic moves along the road. Then as different vehicles pass through the wave, they temporarily experience the higher densities/low speeds which correspond to the wave. Traffic waves move with their own speeds, (typically negative ones, i.e. against the flow of traffic) and the speed at which a traffic wave moves is separate from the speed of the vehicles. <br> <br>
Traffic oscillations should be thought of in the context of equilibrium solutions. Recall that in the equilibrium solution, vehicles have a constant speed. Meaning, if one took a vehicle in equilibrium, and plotted its trajectory in the speed-headway plane, the trajectory would simply be a point corresponding to the (speed, headway) pair. In contrast, for the trajectory of a vehicle experiencing oscillations, its plot would be some path through the speed-headway plane. Because vehicles are closely coupled to their lead vehicles, it is relevant to think of how the oscillation changes as it passes through vehicles. Specifically, for some lead vehicle in oscillation, how does the speed-headway plot of the following vehicle compare to that of the leader? If a car following model is 'string stable', then the follower's speed-headway plot will be closer to some equilibrium solution. After passing through enough vehicles (i.e. the follower, follower's follower, etc.) the speed-headway plot will simply be some single point of an equilibrium solution. If the car following model is 'string unstable', then the opposite happens, and the curves/path in the speed-headway plane become more and more pronounced as the oscillation passes through vehicles.

# The vehicle class
The vehicle class is most fundamental part of the simulation module. It dictates the logic of how vehicles behave. Its basic functions are:
- Handling calls to the car following and lane changing models (set_cf and set_lc). It also has any extra methods related to those models (e.g. eqlfun, free_cf, shift_eql methods).
- Updates the vehicle's longitudinal state given the output from the car following model (update method)
- Maintains memory of past positions, speeds, and any other memory of past quantities that is necessary for computations i.e. computing the gradient
- Maintains leader/follower orders necessary for the lane changing model, i.e. has updated values for lead, fol, lfol, rfol, llead, rlead attributes
- Vehicles have to travel on roads in a way which is consistent with the network and also ensures they follow their own prespecfied route through the network (i.e. they keep the lane_events and route_events attributes)

### Customizing vehicles
Currently the Vehicle class is implemented for a second order ODE model. The way to implement your own car following model of this type is to inherit Vehicle, and override cf_model. <br>
Similarly, to implement your own lane changing model, override set_lc. Using set_lc_helper in your own custom version will probably be helpful. <br>
Note Vehicle parameters are set by new_vehicle, which is a method for any Lanes with upstream boundary conditions. <br>
Although just overriding cf_model and set_lc_helper is enough, you may want to override some other methods as well. eqlfun is needed for the 'speed' and 'congested' options for get_inflow and for the 'ceql' and 'feql' methods for increment_inflow. It is also needed for the 'flow' method for get_inflow because of the edge case when self.anchor.lead is None in increment_inflow.  These boundary conditions require eqlfun because they use the get_eql, get_flow, and inv_flow methods of Vehicle. get_flow and get_eql are also useful in general, i.e. to analyze the behavior of vehicles. <br> free_cf is needed for the 'free' and 'free merge' options for get_downstream. Calculating eqlfun and free_cf is typically not difficult. <br> shift_eql is used for lane changing if the tactical or cooperative model are being used. This can be more tricky to define than the free_cf or eqlfun methods, but if it is possible to derive eqlfun analytically, it should be possible to derive shift_eql as well. It is also possible to use a simple function for shift_eql, e.g. using a constant acceleration/deceleration instead of adaptively choosing it based on vehicle's speed.
##### Future plans for customization
In the future there will be base classes for different types of models, e.g. 1st, 2nd, 3rd order, time delay or no time delay, stochastic elements or not stochastic elements, parametric or nonparametric model, etc. At present we elaborate on how these sorts of changes fit into the current code.

Implementing  __models with time delay__ is implemented in simulation.circular_route. That code is designed for vehicles having different time delays. Vehicles having different time delays adds some extra overheads to simulation because you have to figure out what order vehicles are evaluated in. It is also not clear how to parralelize in that case. If vehicles have the same time delays, it is much simpler and very similar to the case of no time delays.<br>

Implementing a __first or third order model__ with Vehicle is possible but at the moment is significantly more complicated. First, you will have to override get_cf and set_cf in addition to cf_model so that the call signature for the model is respected. The mobil model used by default for lane changing assumes the model outputs acceleration. In general mobil could be used for a model which outputs jerk/speeds, but the default parameter values will need to be changed. The update method will also need to be changed to be consistent with whatever discretization you want to use. For a third order model you would probably want to change \_\_init\_\_ as well to add memory for acceleration. Lastly, the current upstream/downstream boundary conditions assume a second order model, so increment_inflow and get_downstream will need to be adjusted. <br>

For __models with stochastic components__, this can be implemented using the current Vehicle class. In that case you would want to use the expected value for eqlfun. There are many extra complications for taking the gradient of simulations where the models are stochastic. Adding support for this is very high priority. <br>

For __nonparametric models__ (i.e. neural networks), it is not possible to define eqlfun or free_cf analytically. The question of how to handle those (and the boundary conditions they are used for) properly is an open research question.



### Some Naming conventions
l/r - left and right respectively <br>
position - pos or x (x can also be an input though)<br>
speed - speed, spd, or v (reads as velocity) <br>
acceleration - acc, a <br>
(space) headway - hd, s <br>
time - time, t <br>
time step (time discretization) - dt or h <br>
mem - memory (past values)  <br>
bounds - bounds should be a tuple, or a list of tuples, of min/max values, in that order<br>
lead/fol - leader, follower
eql - equilibrium 
cf - car following
lc - lane changing/lane change



### Notes on vehicle orders (leader/follower relationships)
##### Definitions
Consider some vehicle, which we will name the ego vehicle. The lead vehicle is the vehicle directly in front of the ego vehicle. Similarly, the follower is the vehicle directly behind the ego vehicle. These are called lead/fol in the code. Putting an l or r in front of lead/fol indicates that it is the left/right leader/follower (e.g. llead = left leader). Left means that the vehicle is in the lane directly to the left of the ego vehicle's lane, i.e. the llane. The lfol is the vehicle in the llane which is closest to the ego vehicle, without going past the position of the ego vehicle. Meaning that the distance from the lfol to the ego vehicle is positive, and of all vehicles in the left lane, the lfol has the smallest positive distance to the ego vehicle. There can be only one lfol. llead has two possible meanings. The llead attribute is a set of all vehicles in the left lane which have the ego vehicle as a rfol. When using llead in the context of a lane changing model, we mean the leader of lfol. Note that the leader of lfol is not necessarily in the llead attribute. (Such a situation is pictured below). The rfol and rlead are defined analogously. 
<img src="updatingLC.jpg" style="width: 650px;"/>
In the picture above, boxes are vehicles, i is the ego vehicle. Vertical lines indicate different lanes. i is changing to the left, so the the left side is the 'lane changing side' or lcside for short. The right side is similarly the 'opposite side lane' or opside for short. The vehicle labeled as lcside follower is the lfol for i, the llead attribute for i is empty, the rlead attribute is the opposite side leader, and there is no rfol. After i changes lanes, lcside lead will be the new leader and lcsidefollower will be the new follower. After i changes lanes, its new left lane will be the lane labeled as newlcside (new lane changing side), and it's llead attribute after changing lanes will include both the vehicle labeled as 'new lc leader' as well as the leader of new lc leader.
##### Maintaing vehicle orders - l/rlead, anchors, and merge anchors
We do not explicitly maintain the full vehicle orders, as it is inefficient to do so. Rather, an ego vehicle keeps references to their leader, follower, left and right follower, and also have a set of all vehicles which have the ego vehicle as a left/right follower (the rlead and llead attributes, respectively). The reason why the llead and rlead attributes are necessary to maintain is to ensure that a vehicle's lfol and rfol are updated properly after a lane change. During every timestep, we also check a vehicle's lfol and rfol to see if either of them have overtaken the vehicle (update_lrfol). Note as well that since llead/rlead basically have the 'opposite' of lfol and rfol, everytime you change an lfol/rfol, there must also be an llead/rlead you have to change as well. <br>
Anchors are 'fake' vehicles which do not actually exist and can only their vehicle order attributes (lead, fol, etc.) updated, as they have no lane changing or car following models. They are identified because their cf_parameters attribute is None. All lanes have an anchor associated with it, which represents the beginning of the lane. The anchor has its rlead/llead/lead attributes updated so that vehicles will have the correct vehicle order upon being added. <br>
There is also the concept of merge anchors. Whenever there is a merge (i.e. two lanes which were previously seperate and had no left/right lanes, join so that they have each other as left/right lanes), there needs to be a way to obtain vehicle's left followers in the new lane that becomes available (As left/right leaders are simply reciprocals of the left/right followers, it is only necessary to ensure the l/rfol are set correctly and then to update the l/rlead accordingly). Merge anchors accomplish this, i.e. they give a way to find the new left followers which become available when lanes merge. Since we don't maintain sorted lists of vehicles, instead we maintain a good guess as a vehicle which should be close to the left follower (or is the left follower), and then do a search starting from this good guess. The good guess is the merge anchor, and merge anchors are updated every timestep just like l/rfol are. <br>
In general there are other ways to define vehicle orders. See the block comment above update_change in simulation for some notes on this. Note that in any case, maintaining a sorted list of vehicles is inefficient and can be avoided. 
##### Standard form for lane changing model
For most lane changing models to evaluate whether or not to change lanes (for example, whether or not to change to the left lane), they take as input the current headway, and the 'new headway' they were to have if they changed lanes. In the picture above, the current headway is the headway between i and leader. The new headway is the headway between i and lcside leader. Other inputs may include the current and new headways for the lfol, and fol. Note that the only vehicles which would have their headway changed are lfol, fol, and the ego vehicle. Beside the current/new headways, it is also common to use the speeds of vehicles. lead and llead do not have their headway changed, but their speeds may be still be used in the lane changing model. The default lane changing model mobil uses the new/current headways and the speeds of the ego vehicle, fol, lfol, and it also uses the speeds of lead and lcside lead.
### Differentiating vehicle trajectories
Coming Soon, very high priority. Currently there is an implementation of this in calibration.optimization

# The Lane class
The Lane class is the base class for defining road networks. It's basic functions are 
- Computing headways and distances between vehicles
- Ensures vehicles have the correct left/right connections, and that vehicles transition to the next lane correctly after reaching the end of their current lane
- Maintains references to anchors and merge anchors so that the vehicle order is correctly defined and updated
- Contains the methods for downstream boundary conditions and removing vehicles from the network
- Contains the methods for upstream boundary conditions and adding vehicles to the network

Currently it is quite cumbersome to correctly define lanes and road networks correctly. We are developing a better and much simpler way to define road networks in the future. For the time being, you have to manually specify basically everything, which is error prone and requires an in-depth understanding of the code. Improving this aspect is the other very high priority task in the simulation module, alongside being able to differentiate vehicle trajectories and stochastic vehicle trajectories.

### Lanes, Roads, Tracks
A lane just represents some segment vehicles can travel on. Currently a vehicle's lane is discrete valued - you can't be halfway between lane 0 and 1 - you have to be on one or the other. Lanes have a 'local' coordinate system, meaning that if you are comparing positions between two lanes, you also have to know the distance between the lanes. It's not enough to simply compare the two positions. This is handled by the 'roadlen' attribute all lanes have. roadlen is a dictionary which has road names (strings) as keys, and values as the distance from the self to the road. <br>
The way lanes make sure the vehicles respect the network topology (e.g. have the correct left/right connections, move to the next lane at the correct time) is through 'lane events'. All lanes have an events attribute which keeps track of all such changes. Vehicles have a lane_events attribute which keep track of those events. <br>
A road is a collection of lanes. A lane can only belong to a single road, but it can start and end at whatever positions are convenient. However, a road does have to have a consistent local coordinate system - if you move from one lane to another lane in the same road, you shouldn't have to change your position. In contrast, if you move from one lane to another lane in a different road, you will be in a new coordinate system and need to update your position accordingly. Besides this requirement of a local coordinate system, each lane in a road must have a unique index. Additionally, the index must be so that the leftmost lane is index 0, and the rightmost lane has the largest index. Roads don't have any special objects right now, they are simply a dictionary. <br>
A track is just a series of lanes which share the same anchor, meaning that a vehicle could travel along the entire track without making any lane changes.
### Routes
A route is a series of roads the vehicle wants to travel on. It should end with 'exit', and the road a vehicle enters the road network on is not included in the route. Currently when vehicles are calculating their routes, they do so one road at a time. This means that if roads are very short, the route may be difficult to follow as vehicles don't 'look ahead'. Routes may also behave strangely in the case where some lane which is not the leftmost or rightmost lane does not start in the beginning of the road. For example, say we have a road with 4 lanes, indexed 0-3. Initially, the road has 3 lanes all connected. At some later point, lane 1 splits up into indexes 1 and 2 (a diverge). In this case, for a vehicle which wants to end up on the side of the diverge with lanes 2-3, it will move to the correct side until changing to 2 becomes available. See make_route_helper for some more discussion of this. <br>
Other limitations of routes - after a vehicle leaves its previous road, it cannot return to it. Also, vehicles can only travel on roads in their route. Using the same 4 lane road with a diverge as an example, perhaps 0-1 are considered one road, and 2-3 a second road. A vehicle traveling on the 0-1 road may wish to temporarily change to 2-3 and change back in order to overtake a vehicle. This type of manuever is not currently possible. The last situation we don't currently support is for when vehicles aren't able to follow their routes. In some simulators, vehicles have to come to complete stop and try to follow their routes no matter what, which may lead to vehicles unrealistic blocking roads. In other simulators, vehicles which fail to follow their route are simply removed. Another approach would be to give vehicles a new route. 
### Boundary Conditions
Boundary conditions affect how vehicles can enter/exit the simulation. This has a non trivial affect on how waves/oscillations form and propagate inside the simulation. In particular, 'artifacts' can occur when the upstream/downstream boundaries aren't consistent with the actual conditions inside the simulation. For example, perhaps vehicles are added with too small of a speed, and start accelerating strongly as soon as they are added, or perhaps the opposite situation occurs and vehicles strongly decelerate as soon as they enter. Maybe your downstream boundaries are not set up correctly and cause traffic waves that shouldn't have been formed. 
### Defining a road network
Api to do this is coming soon, along with a road network object. 
Currently the parts that need to be manually specified are:
- the road dictionary itself, including references to the lanes, and the 'connect to' value
- the roadlen dictionary
- anchors, and any merge anchors
- the lane events (events attribute)
- the connect left/right attributes and get_downstream call can typically only be specified after the lane is already created. 
When passing to the Simulation object, you only pass lists of any lanes with a increment_inflow method (inflow_lanes) and any lanes with a merge_anchors attribute (merge_lanes). 


# Example simulation
see bottleneck simulation.py in scripts > spring 2020
### tests for car following/lane changing model
for car following - infinite road.py, for lane changing - test_set_lc.py, both in scripts > spring 2020